In [31]:
import warnings
from math import comb

import numpy as np
import pandas as pd
from scipy.stats import bernoulli, logistic

warnings.filterwarnings("ignore")

In [32]:
# Pooling====
# GROUP TESTING
# CREATED: Eli P. Fenichel, Yale University
# LAST UPDATED: July 15, 2020
# Probability of a groups testing positive accounting for sensitivity of test
# with prevalence m, group size = group, population = pop


def estimate_pooling_prob_of_pos_groups(group_size, m, pop, se, s_loss):
    shat = se - group_size * s_loss
    p_pos = 1 - (1 - shat * m) ** group_size  # shat es la sensibilidad,
    # p.pos: Probabilidad que un grupo sea positivo
    # (1-prevalencia): Probabilidad que se salga negativo una persona
    # (1-prevalencia)^tamano_grupo: Probabilidad que todos sean negativos en el grupo
    return p_pos

In [33]:
# Group results
def estimate_pooling_tests(group_size, p, pop, se, s_loss, sp):
    p_pos_pool = estimate_pooling_prob_of_pos_groups(group_size, p, pop, se, s_loss)
    p_pos_test = se * p + (1 - sp) * (1 - p)
    # number of groups
    n_group = int(np.round(pop / group_size))
    pos_groups = np.sum(bernoulli.rvs(p_pos_pool, size=n_group))
    neg_groups = n_group - pos_groups  # cambio

    # number of people
    neg_peop_group = neg_groups * group_size
    pos_peop_group = pos_groups * group_size

    pos_peop = pos_peop_group * p_pos_test

    n_groups_pooling = pos_groups + neg_groups
    total_tests_pooling = n_groups_pooling + pos_peop
    test_per_person_pooling = total_tests_pooling / pop

    # the expected number of people in negative groups that are actually
    # false positives with group size = group.size, prevalence p, sensitivity s,
    # in a population p.

    shat = se - group_size * s_loss
    t_prob = (1 - shat) * p + (1 - p)
    expected_number_missed = np.zeros(group_size - 1)
    for x in range(group_size - 1):
        expected_number_missed[x] = (
            comb(group_size, group_size - (x + 1))
            * ((1 - p) / t_prob) ** (group_size - (x + 1))
            * ((1 - shat) * p / t_prob) ** (x + 1)
        )

    exp_m = np.matmul(range(1, group_size), expected_number_missed)
    gm = exp_m * neg_groups

    res = pd.DataFrame(
        {
            "p_pos_pool": [p_pos_pool],
            "p_pos_test": [p_pos_test],
            "pos_groups": [pos_groups],
            "neg_groups": [neg_groups],
            "pos_peop_group": [pos_peop_group],
            "pos_peop": [pos_peop],
            "neg_peop_group": [neg_peop_group],
            "n_groups_pooling": [n_groups_pooling],
            "n_undetected_pos": [gm],
            "undetected_groups": [exp_m],
            "total_tests_pooling": [total_tests_pooling],
            "number_test_per_person_pooling": [test_per_person_pooling],
        }
    )
    return res

In [34]:
def covid_testing_strategies(
    prevalence,
    sens_model,
    spec_model,
    sens_ag_test,
    spec_ag_test,
    sens_pcr_test,
    spec_pcr_test,
    sens_lamp_test,
    spec_lamp_test,
    people,
    cost_pcr,
    cost_antigen,
    cost_lamp,
    group_size,
    strategy,
    budget,
):
    df_out = pd.DataFrame({})
    sens = sens_model
    spec = spec_model
    # for sens in sens_model:
    #     for spec in spec_model:

    # Probability that an a test gives a positive result
    # Antigen
    prob_res_pos_ag = sens_ag_test * prevalence + (1 - spec_ag_test) * (1 - prevalence)
    # LAMP
    prob_res_pos_lamp = sens_lamp_test * prevalence + (1 - spec_lamp_test) * (
        1 - prevalence
    )
    # PCR
    prob_res_pos_pcr = sens_pcr_test * prevalence + (1 - spec_pcr_test) * (
        1 - prevalence
    )
    # Probability that a test gives a negative result
    prob_res_neg_ag = 1 - prob_res_pos_ag
    prob_res_neg_lamp = 1 - prob_res_pos_lamp
    prob_res_neg_pcr = 1 - prob_res_pos_pcr

    # Probability model gives a low risk classification
    # prob_model_low_risk = (1 - sens) * prevalence +
    #   spec * (1 - prevalence)
    # prob_model_high_risk = 1 - prob_model_low_risk

    prob_model_high_risk_ag = sens * prob_res_pos_ag + (1 - spec) * prob_res_neg_ag
    # prob_model_high_risk_ag = 1 - prob_model_low_risk_ag

    prob_model_high_risk_pcr = sens * prob_res_pos_pcr + (1 - spec) * prob_res_neg_pcr
    # prob_model_high_risk_pcr = 1 - prob_model_low_risk_pcr

    q_model_high_risk_ag = logistic.ppf(prob_model_high_risk_ag)
    q_model_high_risk_pcr = logistic.ppf(prob_model_high_risk_pcr)

    prob_model_high_risk = logistic.cdf(
        0.5 * (q_model_high_risk_ag + q_model_high_risk_pcr)
    )
    prob_model_low_risk = 1 - prob_model_high_risk

    # PPV y NPV of model
    npv_model_ag = spec * prob_res_neg_ag / prob_model_low_risk
    npv_model_pcr = spec * prob_res_neg_pcr / prob_model_low_risk

    q_npv_model_low_risk_ag = logistic.ppf(npv_model_ag)
    q_npv_model_low_risk_pcr = logistic.ppf(npv_model_pcr)

    npv_model = logistic.cdf(0.5 * (q_npv_model_low_risk_ag + q_npv_model_low_risk_pcr))
    #
    # npv_model <- spec * (1 - prevalence) /
    #   ((1 - sens) * prevalence + spec * (1 - prevalence))

    # People in risk
    people_high_risk = np.sum(bernoulli.rvs(prob_model_high_risk, size=people))
    people_low_risk = people - people_high_risk

    # Probability that given High risk classification
    # the patient has a positive result with antigen

    prob_res_pos_given_high_risk_ag = sens * prob_res_pos_ag / prob_model_high_risk

    prob_res_neg_given_high_risk_ag = (
        (1 - spec) * prob_res_neg_ag / prob_model_high_risk
    )

    prob_res_pos_given_low_risk_ag = (1 - sens) * prob_res_pos_ag / prob_model_low_risk

    prob_res_pos_given_high_risk_lamp = sens * prob_res_pos_lamp / prob_model_high_risk

    prob_res_neg_given_high_risk_lamp = 1 - prob_res_pos_given_high_risk_lamp

    prob_res_pos_given_high_risk_pcr = sens * prob_res_pos_pcr / prob_model_high_risk

    prob_res_neg_given_high_risk_pcr = 1 - prob_res_pos_given_high_risk_pcr

    # Probability that given High risk classification
    # the patient has a negative result with antigen
    prob_res_neg_given_high_risk_ag = (
        (1 - spec) * prob_res_neg_ag / prob_model_high_risk
    )

    # PPV golden test
    ppv_golden_test = (
        sens_pcr_test
        * prevalence
        / (sens_pcr_test * prevalence + (1 - spec_pcr_test * (1 - prevalence)))
    )

    # Strategy 1 -------------------------------------------------------
    if strategy == 1:
        # Probability of people tested after of 5 days since symptoms onset
        for prob_symp_more_5d in [0.25, 0.5, 0.75]:
            # Probability of people tested before of 5 days since symptoms onset.
            prob_symp_less_5d = 1 - prob_symp_more_5d

            #             people_low_risk = 0

            #             # Number of antigen tests done
            #             number_alt_tests = np.sum(
            #                 bernoulli.rvs(prob_symp_less_5d, size=people_high_risk)
            #             )

            #             # Number of PCR tests done
            #             number_pcr_tests = (
            #                 people_high_risk
            #                 - number_alt_tests
            #                 + np.sum(
            #                     bernoulli.rvs(
            #                         prob_res_neg_given_high_risk_ag, size=number_alt_tests
            #                     )
            #                 )
            #             )

            #             number_test_per_person = (number_pcr_tests + number_alt_tests) / (
            #                 people_high_risk
            #             )

            #             number_positive_reported = (
            #                 people_res_pos_given_high_risk_ag
            #                 + people_res_pos_given_res_neg_ag_high_risk
            #                 + people_res_pos_given_high_risk_pcr
            #                 + pooling_tests_low_risk["pos_peop"]
            #             )

            # Number of antigen tests done
            people_symp_less_5d = np.sum(
                bernoulli.rvs(prob_symp_less_5d, size=people_high_risk)
            )
            people_res_pos_given_low_risk_ag = np.sum(
                bernoulli.rvs(prob_res_pos_given_low_risk_ag, size=people_low_risk)
            )

            number_alt_tests = people_symp_less_5d

            # Number of PCR tests done
            people_symp_more_5d = people_high_risk - people_symp_less_5d

            people_res_neg_given_high_risk_ag = np.sum(
                bernoulli.rvs(prob_res_neg_given_high_risk_ag, size=people_symp_less_5d)
            )
            people_res_pos_given_high_risk_ag = (
                people_symp_less_5d - people_res_neg_given_high_risk_ag
            )
            people_res_pos_given_high_risk_pcr = np.sum(
                bernoulli.rvs(
                    prob_res_pos_given_high_risk_pcr, size=people_symp_more_5d
                )
            )

            people_res_pos_given_res_neg_ag_high_risk = np.sum(
                bernoulli.rvs(
                    prob_res_pos_given_high_risk_pcr,
                    size=people_res_neg_given_high_risk_ag,
                )
            )

            number_pcr_tests = (
                people_high_risk
                - number_alt_tests
                + np.sum(
                    bernoulli.rvs(
                        prob_res_neg_given_high_risk_ag, size=number_alt_tests
                    )
                )
            )
            number_test_per_person = (number_pcr_tests + number_alt_tests) / (
                people_high_risk
            )
            # Total costs.
            total_cost_alt = cost_antigen * number_alt_tests
            total_cost_pcr = cost_pcr * number_pcr_tests
            total_cost = total_cost_alt + total_cost_pcr

            number_positive_reported = (
                people_res_pos_given_high_risk_ag
                + people_res_pos_given_res_neg_ag_high_risk
                + people_res_pos_given_high_risk_pcr
            )

            stock_capacity = (
                (number_pcr_tests + number_alt_tests)
                / (total_cost)
                * ((people_high_risk) / people)
            )

            efficiency = (
                number_positive_reported / (total_cost) * ((people_high_risk) / people)
            )

            stock_capacity = budget * float(stock_capacity)
            efficiency = budget * float(efficiency)

            df2 = pd.DataFrame(
                {
                    "strategy": strategy,
                    "prevalence": [prevalence],
                    "people": [people_high_risk + people_low_risk],
                    "people_high_risk": [people_high_risk],
                    "people_low_risk": [people_low_risk],
                    "sens_model": [sens],
                    "spec_model": [spec],
                    "sens_ag_test": [sens_ag_test],
                    "spec_ag_test": [spec_ag_test],
                    "prob_symp_less_5d": [prob_symp_less_5d],
                    "prob_symp_more_5d": [prob_symp_more_5d],
                    "number_alt_tests": [number_alt_tests],
                    "number_pcr_tests": number_pcr_tests,
                    "number_positive_reported": number_positive_reported,
                    "number_positive_theoretical": [people * prevalence],
                    "number_test_per_person": number_test_per_person,
                    "total_cost_alt": [total_cost_alt],
                    "total_cost_pcr": [total_cost_pcr],
                    "total_cost": [total_cost],
                    "stock_capacity": [stock_capacity],
                    "efficiency": [efficiency],
                }
            )

            df_out = pd.concat([df_out, df2])

    elif strategy == 2:
        # Probability of people tested after of 5 days since symptoms onset
        for prob_symp_more_5d in [0.25, 0.5, 0.75]:
            prob_symp_less_5d = 1 - prob_symp_more_5d

            pooling_tests_full_prevalence = pd.DataFrame()

            pooling_tests_full_prevalence = estimate_pooling_tests(
                group_size=group_size,
                p=prevalence,
                pop=people_low_risk,
                se=sens_pcr_test,
                s_loss=0,
                sp=spec_pcr_test,
            )

            pooling_tests_low_risk = estimate_pooling_tests(
                group_size=group_size,
                p=1 - npv_model,
                pop=people_low_risk,
                se=sens_pcr_test,
                s_loss=0,
                sp=spec_pcr_test,
            )

            # Number of antigen tests done
            people_symp_less_5d = np.sum(
                bernoulli.rvs(prob_symp_less_5d, size=people_high_risk)
            )
            people_res_pos_given_low_risk_ag = np.sum(
                bernoulli.rvs(prob_res_pos_given_low_risk_ag, size=people_low_risk)
            )

            number_alt_tests = people_symp_less_5d

            # Number of PCR tests done
            people_symp_more_5d = people_high_risk - people_symp_less_5d

            people_res_neg_given_high_risk_ag = np.sum(
                bernoulli.rvs(prob_res_neg_given_high_risk_ag, size=people_symp_less_5d)
            )
            people_res_pos_given_high_risk_ag = (
                people_symp_less_5d - people_res_neg_given_high_risk_ag
            )
            people_res_pos_given_high_risk_pcr = np.sum(
                bernoulli.rvs(
                    prob_res_pos_given_high_risk_pcr, size=people_symp_more_5d
                )
            )

            people_res_pos_given_res_neg_ag_high_risk = np.sum(
                bernoulli.rvs(
                    prob_res_pos_given_high_risk_pcr,
                    size=people_res_neg_given_high_risk_ag,
                )
            )

            number_pcr_tests = (
                people_high_risk
                - number_alt_tests
                + np.sum(
                    bernoulli.rvs(
                        prob_res_neg_given_high_risk_ag, size=number_alt_tests
                    )
                )
                + pooling_tests_low_risk["total_tests_pooling"].values
            )
            number_test_per_person = (number_pcr_tests + number_alt_tests) / (people)
            # Total costs.
            total_cost_alt = cost_antigen * number_alt_tests
            total_cost_pcr = cost_pcr * number_pcr_tests
            total_cost = total_cost_alt + total_cost_pcr

            number_positive_reported = (
                people_res_pos_given_high_risk_ag
                + people_res_pos_given_res_neg_ag_high_risk
                + people_res_pos_given_high_risk_pcr
                + pooling_tests_low_risk["pos_peop"]
            )

            # number_positive_reported <-
            #   people_high_risk * prob_symp_less_5d *
            #   (
            #     prob_res_pos_given_high_risk_ag +
            #       prob_res_neg_given_high_risk_ag *
            #         prob_res_pos_given_high_risk_pcr
            #   ) +
            #   people_high_risk * prob_symp_more_5d *
            #     prob_res_pos_given_high_risk_pcr +
            #   pooling_tests_low_risk$pos_peop
            # stock_capacity = ((number_pcr_tests + number_alt_tests) / (total_cost)) * (
            #     (people_high_risk + people_low_risk) / people
            # )
            stock_capacity = (number_pcr_tests + number_alt_tests) / (total_cost)

            efficiency = number_positive_reported / (total_cost)

            # efficiency = (number_positive_reported / (total_cost)) * (
            #     (people_high_risk + people_low_risk) / people
            # )

            stock_capacity = budget * float(stock_capacity)
            efficiency = budget * float(efficiency)

            df2 = pd.DataFrame(
                {
                    "strategy": strategy,
                    "prevalence": [prevalence],
                    "people": [people_high_risk + people_low_risk],
                    "people_high_risk": [people_high_risk],
                    "people_low_risk": [people_low_risk],
                    "sens_model": [sens],
                    "spec_model": [spec],
                    "sens_ag_test": [sens_ag_test],
                    "spec_ag_test": [spec_ag_test],
                    "prob_symp_less_5d": [prob_symp_less_5d],
                    "prob_symp_more_5d": [prob_symp_more_5d],
                    "number_alt_tests": [number_alt_tests],
                    "number_pcr_tests": number_pcr_tests,
                    "number_positive_reported": number_positive_reported,
                    "number_positive_theoretical": [people * prevalence],
                    "number_test_per_person": number_test_per_person,
                    "total_cost_alt": total_cost_alt,
                    "total_cost_pcr": total_cost_pcr,
                    "total_cost": total_cost,
                    "stock_capacity": [stock_capacity],
                    "efficiency": [efficiency],
                }
            )

            df_out = pd.concat([df_out, df2])

    elif strategy == 3:
        # Probability of people tested after of 5 days since symptoms onset
        for prob_symp_more_5d in [0.25, 0.5, 0.75]:
            prob_symp_less_5d = 1 - prob_symp_more_5d
            # Number of antigen tests done
            people_symp_less_5d = np.sum(
                bernoulli.rvs(prob_symp_less_5d, size=people_high_risk)
            )
            people_res_pos_given_low_risk_ag = np.sum(
                bernoulli.rvs(prob_res_pos_given_low_risk_ag, size=people_low_risk)
            )

            number_alt_tests = (
                people_symp_less_5d + people_low_risk + people_res_pos_given_low_risk_ag
            )

            # Number of PCR tests done
            people_symp_more_5d = people_high_risk - people_symp_less_5d

            people_res_neg_given_high_risk_ag = np.sum(
                bernoulli.rvs(prob_res_neg_given_high_risk_ag, size=people_symp_less_5d)
            )

            number_pcr_tests = people_symp_more_5d + people_res_neg_given_high_risk_ag

            people_res_pos_given_high_risk_ag = (
                people_symp_less_5d - people_res_neg_given_high_risk_ag
            )
            people_res_pos_given_high_risk_pcr = np.sum(
                bernoulli.rvs(
                    prob_res_pos_given_high_risk_pcr, size=people_symp_more_5d
                )
            )

            people_res_pos_given_res_neg_ag_high_risk = np.sum(
                bernoulli.rvs(
                    prob_res_pos_given_high_risk_pcr,
                    size=people_res_neg_given_high_risk_ag,
                )
            )

            people_confirmed_retest_ag = np.sum(
                bernoulli.rvs(prob_res_pos_given_low_risk_ag**2, size=people_low_risk)
            )

            number_positive_reported = (
                people_res_pos_given_high_risk_ag
                + people_res_pos_given_res_neg_ag_high_risk
                + people_res_pos_given_high_risk_pcr
                + people_confirmed_retest_ag
            )

            number_test_per_person = (number_pcr_tests + number_alt_tests) / (people)
            total_cost_alt = cost_antigen * number_alt_tests
            total_cost_pcr = cost_pcr * number_pcr_tests
            total_cost = total_cost_alt + total_cost_pcr

            stock_capacity = (number_pcr_tests + number_alt_tests) / (total_cost)

            efficiency = number_positive_reported / (total_cost)

            stock_capacity = budget * float(stock_capacity)
            efficiency = budget * float(efficiency)

            df2 = pd.DataFrame(
                {
                    "strategy": strategy,
                    "prevalence": [prevalence],
                    "people": [people_high_risk + people_low_risk],
                    "people_high_risk": [people_high_risk],
                    "people_low_risk": [people_low_risk],
                    "sens_model": [sens],
                    "spec_model": [spec],
                    "sens_ag_test": [sens_ag_test],
                    "spec_ag_test": [spec_ag_test],
                    "prob_symp_less_5d": [prob_symp_less_5d],
                    "prob_symp_more_5d": [prob_symp_more_5d],
                    "number_alt_tests": [number_alt_tests],
                    "number_pcr_tests": number_pcr_tests,
                    "number_positive_reported": number_positive_reported,
                    "number_positive_theoretical": [people * prevalence],
                    "number_test_per_person": number_test_per_person,
                    "total_cost_alt": total_cost_alt,
                    "total_cost_pcr": total_cost_pcr,
                    "total_cost": total_cost,
                    "stock_capacity": [stock_capacity],
                    "efficiency": [efficiency],
                }
            )

            df_out = pd.concat([df_out, df2])

    elif strategy == 4:
        for prob_symp_more_5d in [0.25, 0.5, 0.75]:
            prob_symp_less_5d = 1 - prob_symp_more_5d
            prob_model_first_responder_pcr = 0.01
            people_first_responder = np.sum(
                bernoulli.rvs(prob_model_first_responder_pcr, size=people)
            )
            people_high_risk = np.sum(
                bernoulli.rvs(
                    prob_model_high_risk, size=people - people_first_responder
                )
            )
            people_low_risk = people - people_first_responder - people_high_risk

            # Number of test
            number_pcr_tests = people_first_responder

            number_lamp_tests = (
                np.sum(
                    bernoulli.rvs(
                        prob_res_neg_given_high_risk_lamp, size=people_high_risk
                    )
                )
                + people_high_risk
            )

            number_ag_tests = (
                np.sum(
                    bernoulli.rvs(prob_res_pos_given_low_risk_ag, size=people_low_risk)
                )
                + people_low_risk
            )

            number_alt_tests = number_lamp_tests + number_ag_tests

            Highrisk = np.sum(
                bernoulli.rvs(prob_res_pos_given_high_risk_pcr, size=people_high_risk)
            )

            number_positive_reported = (
                np.sum(bernoulli.rvs(prob_res_pos_pcr, size=people_first_responder))
                + Highrisk
                + np.sum(
                    bernoulli.rvs(1 - prob_res_pos_given_high_risk_pcr, size=Highrisk)
                )
                + np.sum(
                    bernoulli.rvs(
                        prob_res_pos_given_low_risk_ag**2, size=people_low_risk
                    )
                )
            )

            number_test_per_person = (number_pcr_tests + number_alt_tests) / (people)

            total_cost_pcr = cost_pcr * number_pcr_tests
            total_cost_alt = (
                cost_lamp * number_lamp_tests + cost_antigen * number_ag_tests
            )
            total_cost = total_cost_alt + total_cost_pcr

            stock_capacity = (number_pcr_tests + number_alt_tests) / total_cost

            efficiency = number_positive_reported / total_cost

            stock_capacity = budget * float(stock_capacity)
            efficiency = budget * float(efficiency)

            df2 = pd.DataFrame(
                {
                    "strategy": strategy,
                    "prevalence": [prevalence],
                    "people": [
                        people_high_risk + people_low_risk + people_first_responder
                    ],
                    "people_high_risk": [people_high_risk],
                    "people_low_risk": [people_low_risk],
                    "people_first_responder": [people_first_responder],
                    "sens_model": [sens],
                    "spec_model": [spec],
                    "sens_ag_test": [sens_ag_test],
                    "spec_ag_test": [spec_ag_test],
                    "sens_lamp_test": [sens_lamp_test],
                    "spec_lamp_test": [spec_lamp_test],
                    "prob_model_first_responder_pcr,": [prob_model_first_responder_pcr],
                    "prob_symp_less_5d": [prob_symp_less_5d],
                    "prob_symp_more_5d": [prob_symp_more_5d],
                    "number_alt_tests": [number_alt_tests],
                    "number_pcr_tests": number_pcr_tests,
                    "number_positive_reported": number_positive_reported,
                    "number_positive_theoretical": [people * prevalence],
                    "number_test_per_person": number_test_per_person,
                    "total_cost_alt": total_cost_alt,
                    "total_cost_pcr": total_cost_pcr,
                    "total_cost": total_cost,
                    "stock_capacity": [stock_capacity],
                    "efficiency": [efficiency],
                }
            )
            df_out = pd.concat([df_out, df2])

    return df_out

In [35]:
def r_covid_testing_strategies(
    prevalence,
    sens_model,
    spec_model,
    strategy,
    people,
    n_simul=1000,
    sens_ag_test=0.8,
    spec_ag_test=0.95,
    sens_pcr_test=0.95,
    spec_pcr_test=0.95,
    sens_lamp_test=0.95,
    spec_lamp_test=0.95,
    cost_pcr=100,
    cost_antigen=50,
    cost_lamp=20,
    group_size=5,
    budget=100000,
):
    resultados = pd.DataFrame()
    for s in range(n_simul):
        df = covid_testing_strategies(
            prevalence,
            sens_model,
            spec_model,
            sens_ag_test,
            spec_ag_test,
            sens_pcr_test,
            spec_pcr_test,
            sens_lamp_test,
            spec_lamp_test,
            people,
            cost_pcr,
            cost_antigen,
            cost_lamp,
            group_size,
            strategy,
            budget,
        )
        df["nsimul"] = s + 1
        df["cost_pcr"] = cost_pcr
        df["cost_antigen"] = cost_antigen
        df["cost_lamp"] = cost_lamp
        resultados = pd.concat([resultados, df], ignore_index=True)
    return resultados

In [36]:
model_parameters = pd.read_pickle(filepath_or_buffer="data/model_parameters.pkl")

In [37]:
df_strategy_1 = r_covid_testing_strategies(
    strategy=1,
    people=1000,
    prevalence=model_parameters.prevalence[0],
    sens_model=model_parameters.sens_model[0],
    spec_model=model_parameters.spec_model[0],
)

df_strategy_1.to_pickle(path="data/df_strategy_1.pkl")
df_strategy_1.head(n=12)

,strategy,prevalence,people,people_high_risk,people_low_risk,sens_model,spec_model,sens_ag_test,spec_ag_test,prob_symp_less_5d,...,number_test_per_person,total_cost_alt,total_cost_pcr,total_cost,stock_capacity,efficiency,nsimul,cost_pcr,cost_antigen,cost_lamp
0,1,0.269096,1000,408,592,0.796296,0.767918,0.8,0.95,0.75,...,1.357843,16100,23200,39300,575.145038,334.290076,1,100,50,20
1,1,0.269096,1000,408,592,0.796296,0.767918,0.8,0.95,0.50,...,1.213235,10400,28700,39100,516.521739,294.260870,1,100,50,20
2,1,0.269096,1000,408,592,0.796296,0.767918,0.8,0.95,0.25,...,1.098039,4700,35400,40100,455.820449,288.957606,1,100,50,20
3,1,0.269096,1000,379,621,0.796296,0.767918,0.8,0.95,0.75,...,1.358839,14400,22700,37100,526.105121,286.037736,2,100,50,20
4,1,0.269096,1000,379,621,0.796296,0.767918,0.8,0.95,0.50,...,1.218997,10050,26100,36150,484.365145,280.973721,2,100,50,20
5,1,0.269096,1000,379,621,0.796296,0.767918,0.8,0.95,0.25,...,1.100264,4200,33300,37500,421.448000,247.613333,2,100,50,20
6,1,0.269096,1000,400,600,0.796296,0.767918,0.8,0.95,0.75,...,1.357500,14750,24800,39550,549.178255,313.527181,3,100,50,20
7,1,0.269096,1000,400,600,0.796296,0.767918,0.8,0.95,0.50,...,1.202500,9400,29300,38700,497.157623,278.036176,3,100,50,20
8,1,0.269096,1000,400,600,0.796296,0.767918,0.8,0.95,0.25,...,1.110000,5350,33700,39050,454.801536,273.495519,3,100,50,20
9,1,0.269096,1000,375,625,0.796296,0.767918,0.8,0.95,0.75,...,1.320000,13650,22200,35850,517.782427,292.887029,4,100,50,20


In [38]:
df_strategy_2 = r_covid_testing_strategies(
    strategy=2,
    people=1000,
    prevalence=model_parameters.prevalence[0],
    sens_model=model_parameters.sens_model[0],
    spec_model=model_parameters.spec_model[0],
)

df_strategy_2.to_pickle(path="data/df_strategy_2.pkl")
df_strategy_2.head(n=12)

,strategy,prevalence,people,people_high_risk,people_low_risk,sens_model,spec_model,sens_ag_test,spec_ag_test,prob_symp_less_5d,...,number_test_per_person,total_cost_alt,total_cost_pcr,total_cost,stock_capacity,efficiency,nsimul,cost_pcr,cost_antigen,cost_lamp
0,2,0.269096,1000,363,637,0.796296,0.767918,0.8,0.95,0.75,...,0.662588,14350,37558.775302,51908.775302,1276.446514,604.113180,1,100,50,20
1,2,0.269096,1000,363,637,0.796296,0.767918,0.8,0.95,0.50,...,0.585380,8800,40937.987476,49737.987476,1176.927143,565.724286,1,100,50,20
2,2,0.269096,1000,363,637,0.796296,0.767918,0.8,0.95,0.25,...,0.568871,4900,47087.090432,51987.090432,1094.254169,505.646502,1,100,50,20
3,2,0.269096,1000,445,555,0.796296,0.767918,0.8,0.95,0.75,...,0.719247,16600,38724.726955,55324.726955,1300.046668,649.343050,2,100,50,20
4,2,0.269096,1000,445,555,0.796296,0.767918,0.8,0.95,0.50,...,0.676097,10950,45709.672346,56659.672346,1193.259148,630.248480,2,100,50,20
5,2,0.269096,1000,445,555,0.796296,0.767918,0.8,0.95,0.25,...,0.633305,5700,51930.460172,57630.460172,1098.906030,545.379303,2,100,50,20
6,2,0.269096,1000,392,608,0.796296,0.767918,0.8,0.95,0.75,...,0.655588,14000,37558.775302,51558.775302,1271.534766,664.460610,3,100,50,20
7,2,0.269096,1000,392,608,0.796296,0.767918,0.8,0.95,0.50,...,0.621946,9250,43694.617737,52944.617737,1174.710866,557.084346,3,100,50,20
8,2,0.269096,1000,392,608,0.796296,0.767918,0.8,0.95,0.25,...,0.598588,5350,49158.775302,54508.775302,1098.149334,525.764432,3,100,50,20
9,2,0.269096,1000,386,614,0.796296,0.767918,0.8,0.95,0.75,...,0.677305,14650,38430.460172,53080.460172,1275.996100,607.200090,4,100,50,20


In [39]:
df_strategy_3 = r_covid_testing_strategies(
    strategy=3,
    people=1000,
    prevalence=model_parameters.prevalence[0],
    sens_model=model_parameters.sens_model[0],
    spec_model=model_parameters.spec_model[0],
)

df_strategy_3.to_pickle(path="data/df_strategy_3.pkl")
df_strategy_3.head(n=12)

,strategy,prevalence,people,people_high_risk,people_low_risk,sens_model,spec_model,sens_ag_test,spec_ag_test,prob_symp_less_5d,...,number_test_per_person,total_cost_alt,total_cost_pcr,total_cost,stock_capacity,efficiency,nsimul,cost_pcr,cost_antigen,cost_lamp
0,3,0.269096,1000,366,634,0.796296,0.767918,0.8,0.95,0.75,...,1.183,47800,22700,70500,1678.014184,382.978723,1,100,50,20
1,3,0.269096,1000,366,634,0.796296,0.767918,0.8,0.95,0.50,...,1.129,43600,25700,69300,1629.148629,363.636364,1,100,50,20
2,3,0.269096,1000,366,634,0.796296,0.767918,0.8,0.95,0.25,...,1.100,38600,32800,71400,1540.616246,322.128852,1,100,50,20
3,3,0.269096,1000,372,628,0.796296,0.767918,0.8,0.95,0.75,...,1.202,48250,23700,71950,1670.604587,391.938846,2,100,50,20
4,3,0.269096,1000,372,628,0.796296,0.767918,0.8,0.95,0.50,...,1.143,43750,26800,70550,1620.127569,377.037562,2,100,50,20
5,3,0.269096,1000,372,628,0.796296,0.767918,0.8,0.95,0.25,...,1.095,38900,31700,70600,1550.991501,324.362606,2,100,50,20
6,3,0.269096,1000,372,628,0.796296,0.767918,0.8,0.95,0.75,...,1.182,48300,21600,69900,1690.987124,413.447783,3,100,50,20
7,3,0.269096,1000,372,628,0.796296,0.767918,0.8,0.95,0.50,...,1.122,42500,27200,69700,1609.756098,383.070301,3,100,50,20
8,3,0.269096,1000,372,628,0.796296,0.767918,0.8,0.95,0.25,...,1.073,37700,31900,69600,1541.666667,353.448276,3,100,50,20
9,3,0.269096,1000,375,625,0.796296,0.767918,0.8,0.95,0.75,...,1.180,47350,23300,70650,1670.205237,401.981599,4,100,50,20


In [40]:
df_strategy_4 = r_covid_testing_strategies(
    strategy=4,
    people=1000,
    prevalence=model_parameters.prevalence[0],
    sens_model=model_parameters.sens_model[0],
    spec_model=model_parameters.spec_model[0],
)

df_strategy_4.to_pickle(path="data/df_strategy_4.pkl")
df_strategy_4.head(n=12)

,strategy,prevalence,people,people_high_risk,people_low_risk,people_first_responder,sens_model,spec_model,sens_ag_test,spec_ag_test,...,number_test_per_person,total_cost_alt,total_cost_pcr,total_cost,stock_capacity,efficiency,nsimul,cost_pcr,cost_antigen,cost_lamp
0,4,0.269096,1000,382,605,13,0.796296,0.767918,0.8,0.95,...,1.216,44100,1300,45400,2678.414097,709.251101,1,100,50,20
1,4,0.269096,1000,354,641,5,0.796296,0.767918,0.8,0.95,...,1.214,45180,500,45680,2657.618214,639.229422,1,100,50,20
2,4,0.269096,1000,391,600,9,0.796296,0.767918,0.8,0.95,...,1.204,43370,900,44270,2719.674723,729.613734,1,100,50,20
3,4,0.269096,1000,371,622,7,0.796296,0.767918,0.8,0.95,...,1.202,44330,700,45030,2669.331557,699.533644,2,100,50,20
4,4,0.269096,1000,397,594,9,0.796296,0.767918,0.8,0.95,...,1.219,43580,900,44480,2740.557554,768.884892,2,100,50,20
5,4,0.269096,1000,410,583,7,0.796296,0.767918,0.8,0.95,...,1.204,42690,700,43390,2774.832911,795.114082,2,100,50,20
6,4,0.269096,1000,361,626,13,0.796296,0.767918,0.8,0.95,...,1.186,43710,1300,45010,2634.970007,715.396579,3,100,50,20
7,4,0.269096,1000,381,616,3,0.796296,0.767918,0.8,0.95,...,1.201,43880,300,44180,2718.424627,760.525124,3,100,50,20
8,4,0.269096,1000,377,615,8,0.796296,0.767918,0.8,0.95,...,1.190,43680,800,44480,2675.359712,789.118705,3,100,50,20
9,4,0.269096,1000,389,605,6,0.796296,0.767918,0.8,0.95,...,1.192,43220,600,43820,2720.219078,746.234596,4,100,50,20


In [41]:
# Sensitivity analysis for the price of lamp and antigen tests

# lamp_cost_range = np.arange(20, 100, 20)
# antigen_cost_range = np.arange(50, 150, 25)

# df_all_strategy_sensitivity = pd.DataFrame()

# for lamp_cost in lamp_cost_range:
#     for antigen_cost in antigen_cost_range:
#         df_1 = r_covid_testing_strategies(
#             strategy=1,
#             people=1000,
#             prevalence=model_parameters.prevalence[0],
#             sens_model=model_parameters.sens_model[0],
#             spec_model=model_parameters.spec_model[0],
#             cost_lamp=lamp_cost,
#             cost_antigen=antigen_cost,
#         )
#         df_2 = r_covid_testing_strategies(
#             strategy=2,
#             people=1000,
#             prevalence=model_parameters.prevalence[0],
#             sens_model=model_parameters.sens_model[0],
#             spec_model=model_parameters.spec_model[0],
#             cost_lamp=lamp_cost,
#             cost_antigen=antigen_cost,
#         )
#         df_3 = r_covid_testing_strategies(
#             strategy=3,
#             people=1000,
#             prevalence=model_parameters.prevalence[0],
#             sens_model=model_parameters.sens_model[0],
#             spec_model=model_parameters.spec_model[0],
#             cost_lamp=lamp_cost,
#             cost_antigen=antigen_cost,
#         )
#         df_4 = r_covid_testing_strategies(
#             strategy=4,
#             people=1000,
#             prevalence=model_parameters.prevalence[0],
#             sens_model=model_parameters.sens_model[0],
#             spec_model=model_parameters.spec_model[0],
#             cost_lamp=lamp_cost,
#             cost_antigen=antigen_cost,
#         )
#         df_all_strategy_sensitivity = pd.concat(
#             [df_all_strategy_sensitivity, df_1, df_2, df_3, df_4], ignore_index=True
#         )


In [42]:
# df_all_strategy_sensitivity
# df_all_strategy_sensitivity.to_pickle(path="data/df_all_strategy_sensitivity.pkl")
# df_all_strategy_sensitivity.head(n=12)